In [4]:
import pickle
from pkg_resources import resource_filename
from coddiwomple.openmm.coddiwomple import endstate_equilibration
from perses.annihilation.lambda_protocol import RelativeAlchemicalState
import os
from simtk import unit

RDKit WARNING: [17:19:06] Enabling RDKit 2019.09.3 jupyter extensions


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


In [5]:
factory_pickle = f"/data/perses_data/benzene_fluorobenzene.vacuum.factory.pkl"
_pickl = resource_filename('coddiwomple', factory_pickle)
with open(_pickl, 'rb') as f:
    factory = pickle.load(f)


In [6]:
system = factory.hybrid_system

In [7]:
#os.system(f"rm -r tester")

In [8]:
#vacuum equilibration
endstate_equilibration(system = factory.hybrid_system,
                           endstate_positions = factory.hybrid_positions,
                           box_vectors = factory.hybrid_system.getDefaultPeriodicBoxVectors(),
                           directory_name = 'tester',
                           trajectory_prefix = 'eq',
                           md_topology = factory.hybrid_topology,
                           number_of_applications = 10,
                           steps_per_application = 500,
                           endstate_parameters = 0.0,
                           alchemical_composability = RelativeAlchemicalState)

DEBUG:openmm_states:successfully instantiated OpenMMPDFState equipped with the following parameters: {'lambda_sterics_delete': 0.0, 'lambda_electrostatics_core': 0.0, 'lambda_sterics_insert': 0.0, 'lambda_bonds': 0.0, 'lambda_sterics_core': 0.0, 'lambda_torsions': 0.0, 'lambda_sterics': None, 'lambda_electrostatics': None, 'lambda_electrostatics_insert': 0.0, 'lambda_angles': 0.0, 'lambda_electrostatics_delete': 0.0}
DEBUG:openmm_states:successfully updated OpenMMPDFState parameters as follows: {'lambda_sterics_delete': 0.0, 'lambda_electrostatics_core': 0.0, 'lambda_sterics_insert': 0.0, 'lambda_bonds': 0.0, 'lambda_sterics_core': 0.0, 'lambda_torsions': 0.0, 'lambda_electrostatics_insert': 0.0, 'lambda_angles': 0.0, 'lambda_electrostatics_delete': 0.0}
DEBUG:openmm_propagators:initializing OMMLI...
DEBUG:openmm_propagators:OMMLI: metropolization is False
DEBUG:openmm_propagators:OMMLI: successfully parsed splitting string
DEBUG:openmm_propagators:OMMLI: adding global variables...
DEB

step      0 : allow forces to update the context state
step      1 : if(has_kT_changed = 1):
step      2 :    sigma <- sqrt(kT/m)
step      3 :    has_kT_changed <- 0
step      4 : end
step      5 : old_ke <- sum(0.5 * m * v * v)
step      6 : v <- v + (dt / 2) * f / m
step      7 : constrain velocities
step      8 : new_ke <- sum(0.5 * m * v * v)
step      9 : shadow_work <- shadow_work + (new_ke - old_ke)
step     10 : old_pe <- energy
step     11 : old_ke <- sum(0.5 * m * v * v)
step     12 : x <- x + ((dt / 2) * v)
step     13 : x1 <- x
step     14 : constrain positions
step     15 : v <- v + ((x - x1) / (dt / 2))
step     16 : constrain velocities
step     17 : new_pe <- energy
step     18 : new_ke <- sum(0.5 * m * v * v)
step     19 : shadow_work <- shadow_work + (new_ke + new_pe) - (old_ke + old_pe)
step     20 : old_ke <- sum(0.5 * m * v * v)
step     21 : v <- (a * v) + (b * sigma * gaussian)
step     22 : constrain velocities
step     23 : new_ke <- sum(0.5 * m * v * v)
step 

100%|██████████| 10/10 [00:16<00:00,  1.68s/it]

t0: 0, g: 1.0, Neff_max: 11.0, A_t: [40.58818338 39.48585871 36.66560877 48.40918151 38.27617938 50.86961869
 51.75559944 36.18576339 45.24222661 43.85241955], uncorrelated_indices:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
MOL1-C1



Exception ignored in: <bound method PDBTrajectoryFile.__del__ of <mdtraj.formats.pdb.pdbfile.PDBTrajectoryFile object at 0x7fcc209cb588>>
Traceback (most recent call last):
  File "/home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/mdtraj/formats/pdb/pdbfile.py", line 666, in __del__
    self.close()
  File "/home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/mdtraj/formats/pdb/pdbfile.py", line 501, in close
    self._write_footer()
  File "/home/dominic/anaconda3/envs/openmm/lib/python3.6/site-packages/mdtraj/formats/pdb/pdbfile.py", line 429, in _write_footer
    index1 = atomIndex[atom1]
KeyError: (MOL1-C1,)


# let's see if we can make an SMC integrator

In [1]:
default_functions = {'lambda_sterics_core':
                     lambda x: x,
                     'lambda_electrostatics_core':
                     lambda x: x,
                     'lambda_sterics_insert':
                     lambda x: 2.0 * x if x < 0.5 else 1.0,
                     'lambda_sterics_delete':
                     lambda x: 0.0 if x < 0.5 else 2.0 * (x - 0.5),
                     'lambda_electrostatics_insert':
                     lambda x: 0.0 if x < 0.5 else 2.0 * (x - 0.5),
                     'lambda_electrostatics_delete':
                     lambda x: 2.0 * x if x < 0.5 else 1.0,
                     'lambda_bonds':
                     lambda x: x,
                     'lambda_angles':
                     lambda x: x,
                     'lambda_torsions':
                     lambda x: x
                     }

In [2]:
lambda_sequence = {key: [default_functions[key](x) for x in range(500)] for key in default_functions.keys()}

In [ ]:
endstate_cache_filename = 'tester'
num_particles=10

#determine pressure
forces = {type(force).__name__: force for force in system.getForces()}
if "MonteCarloBarostat" in list(forces.keys()):
    pressure = 1.0 * unit.atmosphere
else:
    pressure = None

print(f"pressure: {pressure}")

traj = md.Trajectory.load(endstate_cache_filename)

num_frames = traj.n_frames
pdf_state = OpenMMPDFState(system = system, alchemical_composability = alchemical_composability, pressure=pressure)

#set the pdf_state endstate parameters
pdf_state_parameters = pdf_state.get_parameters()
reset_pdf_state_parameters = {key: endstate_parameters for key in pdf_state_parameters.keys()}
pdf_state.set_parameters(reset_pdf_state_parameters)

#make a target and proposal distribution
target_factory = TargetFactory(pdf_state = pdf_state, parameter_sequence = lambda_sequence, termination_parameters = None)
integrator = OMMLI()
propagator = OMMMBIP(pdf_state, integrator)
proposal_factory = ProposalFactory(parameter_sequence = lambda_sequence, propagator=propagator)

#make a reporter
reporter = OpenMMReporter(directory_name, trajectory_prefix, md_topology = md_topology)

#make an integrator and equip a propagator
integrator = OMMLI()
propagator = OMMMBIP(pdf_state, integrator)

#make a bunch of particles
particles=[Particle(q) for q in range(num_particles)]
for q in range(10):
    particle_state = 



In [ ]:
def SMC_3323()